In [6]:
from functools import reduce, wraps

# Data awal (diinisialisasi di luar fungsi bisnis)
accounts = {}  # Key: NIM, Value: {"password": pass, "role": role}
personnel_data = []  # List of Dictionaries: {"nim": str, "name": str, "rank": str, "unit_id": int}
unit_data = []  # {"unit_id": int, "unit_name": str}
mission_data = []  # (mission_id, mission_name, location, [involved_personnel])

# ======================
# Decorators (Requirement 2)
# ======================
def log_action(action):
    """Decorator to log actions performed by functions."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            print(f"[LOG] {action}: Success")
            return result
        return wrapper
    return decorator

# ======================
# Fungsi (Pure Functions)
# ======================

@log_action("Register User")
def register(accounts, nim, password, role):
    if nim in accounts:
        return accounts, "NIM already registered!"
    new_accounts = {**accounts, nim: {"password": password, "role": role}}
    return new_accounts, "Account successfully registered!"

@log_action("Add Profile")
def add_profile(personnel_data, nim, name, rank, unit_id, unit_data):
    if any(p["nim"] == nim for p in personnel_data):
        return personnel_data, f"Profile for NIM {nim} already exists."

    if unit_id != 0 and not any(u["unit_id"] == unit_id for u in unit_data):
        return personnel_data, f"Unit ID {unit_id} not found. Profile not saved."

    new_profile = {"nim": nim, "name": name, "rank": rank, "unit_id": unit_id}
    new_personnel_data = [*personnel_data, new_profile]
    return new_personnel_data, f"Profile for NIM {nim} has been added."

@log_action("User Login")
def login(accounts, nim, password):
    if nim not in accounts:
        return None, "NIM not found. Please register first."
    if accounts[nim]["password"] == password:
        return nim, f"Welcome, {nim}!"
    else:
        return None, "Incorrect password."

@log_action("View Profile")
def view_profile(personnel_data, unit_data, nim):
    profile = next((p for p in personnel_data if p["nim"] == nim), None)
    if not profile:
        return f"No profile found for NIM {nim}. Please update your profile."

    unit = next((u for u in unit_data if u["unit_id"] == profile["unit_id"]), {"unit_name": "Unknown"})
    return f"Name: {profile['name']}, Rank: {profile['rank']}, Unit: {unit['unit_name']}"

@log_action("Update Profile")
def update_profile(personnel_data, nim, new_name, new_rank, new_unit_id, unit_data):
    if new_unit_id and not any(u["unit_id"] == new_unit_id for u in unit_data):
        return personnel_data, f"Unit ID {new_unit_id} not found. Update cancelled."

    def update(p):
        return {
            "nim": p["nim"],
            "name": new_name or p["name"],
            "rank": new_rank or p["rank"],
            "unit_id": new_unit_id if new_unit_id is not None else p["unit_id"]
        } if p["nim"] == nim else p

    updated_data = list(map(update, personnel_data))
    if updated_data == personnel_data:
        return personnel_data, f"No profile found for NIM {nim}."
    return updated_data, "Profile updated successfully."

@log_action("Delete Profile")
def delete_profile(personnel_data, accounts, nim):
    return (
        [p for p in personnel_data if p["nim"] != nim],
        {k: v for k, v in accounts.items() if k != nim},
        f"Profile and account for NIM {nim} deleted successfully."
    )

# ======================
# Higher Order Functions (Requirement 1)
# ======================

def filter_profiles_by_unit(personnel_data, unit_id):
    """Higher Order Function using lambda to filter profiles by unit."""
    return list(filter(lambda p: p["unit_id"] == unit_id, personnel_data))

def reduce_profiles_by_rank(personnel_data):
    """Reduce operation to count personnel by rank."""
    return reduce(lambda acc, p: {**acc, p["rank"]: acc.get(p["rank"], 0) + 1}, personnel_data, {})

# ======================
# Closure Implementation (Requirement 3)
# ======================

def make_profile_viewer(unit_data):
    """Closure to provide a customized profile viewer with unit information."""
    def view_profile_with_units(nim, personnel_data):
        profile = next((p for p in personnel_data if p["nim"] == nim), None)
        if not profile:
            return f"No profile found for NIM {nim}."

        unit = next((u for u in unit_data if u["unit_id"] == profile["unit_id"]), {"unit_name": "Unknown"})
        return f"Name: {profile['name']}, Rank: {profile['rank']}, Unit: {unit['unit_name']}"

    return view_profile_with_units

# Create a closure instance
view_profile_with_units = make_profile_viewer(unit_data)

# ======================
# Contoh Testing
# ======================
if __name__ == "__main__":
    # Register an account
    accounts, msg = register(accounts, "12345", "password123", "user")
    print(msg)

    # Add unit
    unit_data, msg = add_unit(unit_data, 1, "Alpha Unit")
    print(msg)

    # Add profile
    personnel_data, msg = add_profile(personnel_data, "12345", "John Doe", "Captain", 1, unit_data)
    print(msg)

    # View profile using closure
    print(view_profile_with_units("12345", personnel_data))

    # Filter profiles by unit
    filtered_profiles = filter_profiles_by_unit(personnel_data, 1)
    print(f"Filtered profiles: {filtered_profiles}")

    # Reduce profiles by rank
    rank_summary = reduce_profiles_by_rank(personnel_data)
    print(f"Rank summary: {rank_summary}")


[LOG] Register User: Success
Account successfully registered!
Function add_unit called with args ([], 1, 'Alpha Unit') kwargs {} -> ([{'unit_id': 1, 'unit_name': 'Alpha Unit'}], 'Unit Alpha Unit added successfully.')
Unit Alpha Unit added successfully.
[LOG] Add Profile: Success
Profile for NIM 12345 has been added.
Name: John Doe, Rank: Captain, Unit: Unknown
Filtered profiles: [{'nim': '12345', 'name': 'John Doe', 'rank': 'Captain', 'unit_id': 1}]
Rank summary: {'Captain': 1}


In [7]:
from functools import reduce

# Data awal (diinisialisasi di luar fungsi bisnis)
accounts = {}  # Key: NIM, Value: {"password": pass, "role": role}
personnel_data = []  # List of Dictionaries: {"nim": str, "name": str, "rank": str, "unit_id": int}
unit_data = []  # {"unit_id": int, "unit_name": str}
mission_data = []  # (mission_id, mission_name, location, [involved_personnel])

# ======================
# Fungsi (Pure Functions)
# ======================

def register(accounts, nim, password, role):
    if nim in accounts:
        return accounts, "NIM already registered!"
    new_accounts = {**accounts, nim: {"password": password, "role": role}}
    return new_accounts, "Account successfully registered!"

def add_profile(personnel_data, nim, name, rank, unit_id, unit_data):
    if any(p["nim"] == nim for p in personnel_data):
        return personnel_data, f"Profile for NIM {nim} already exists."

    if unit_id != 0 and not any(u["unit_id"] == unit_id for u in unit_data):
        return personnel_data, f"Unit ID {unit_id} not found. Profile not saved."

    new_profile = {"nim": nim, "name": name, "rank": rank, "unit_id": unit_id}
    new_personnel_data = [*personnel_data, new_profile]
    
    return new_personnel_data, f"Profile for NIM {nim} has been added."

def login(accounts, nim, password):
    if nim not in accounts:
        return None, "NIM not found. Please register first."
    if accounts[nim]["password"] == password:
        return nim, f"Welcome, {nim}!"
    else:
        return None, "Incorrect password."

def view_profile(personnel_data, unit_data, nim):
    profile = next((p for p in personnel_data if p["nim"] == nim), None)
    if not profile:
        return f"No profile found for NIM {nim}. Please update your profile."

    unit = next((u for u in unit_data if u["unit_id"] == profile["unit_id"]), {"unit_name": "Unknown"})
    return f"Name: {profile['name']}, Rank: {profile['rank']}, Unit: {unit['unit_name']}"

def update_profile(personnel_data, nim, new_name, new_rank, new_unit_id, unit_data):
    if new_unit_id and not any(u["unit_id"] == new_unit_id for u in unit_data):
        return personnel_data, f"Unit ID {new_unit_id} not found. Update cancelled."

    def update(p):
        return {
            "nim": p["nim"],
            "name": new_name or p["name"],
            "rank": new_rank or p["rank"],
            "unit_id": new_unit_id if new_unit_id is not None else p["unit_id"]
        } if p["nim"] == nim else p

    updated_data = list(map(update, personnel_data))
    if updated_data == personnel_data:
        return personnel_data, f"No profile found for NIM {nim}."
    return updated_data, "Profile updated successfully."

def delete_profile(personnel_data, accounts, nim):
    return (
        [p for p in personnel_data if p["nim"] != nim],
        {k: v for k, v in accounts.items() if k != nim},
        f"Profile and account for NIM {nim} deleted successfully."
    )

def add_unit(unit_data, unit_id, unit_name):
    if any(u["unit_id"] == unit_id for u in unit_data):
        return unit_data, f"Unit ID {unit_id} already exists."
    new_unit_data = unit_data + [{"unit_id": unit_id, "unit_name": unit_name}]
    return new_unit_data, f"Unit {unit_name} added successfully."

def add_mission(mission_data, mission_id, mission_name, location, involved_personnel):
    new_mission = (mission_id, mission_name, location, involved_personnel)
    new_mission_data = mission_data + [new_mission]
    return new_mission_data, f"Mission {mission_name} added successfully."

def delete_unit(unit_data, unit_id):
    new_unit_data = [u for u in unit_data if u["unit_id"] != unit_id]
    return new_unit_data, f"Unit ID {unit_id} deleted successfully."

def delete_mission(mission_data, mission_id):
    new_mission_data = [m for m in mission_data if m[0] != mission_id]
    return new_mission_data, f"Mission ID {mission_id} deleted successfully."

# ======================
# Wrapper Fungsi I/O
# ======================

# Decorator untuk logging
def log_operation(func):
    def wrapper(*args, **kwargs):
        print(f"Operation: {func.__name__} is being executed.")
        result = func(*args, **kwargs)
        print(f"Operation {func.__name__} completed.")
        return result
    return wrapper

@log_operation
def register_wrapper(accounts):
    nim = input("Enter your NIM: ")
    password = input("Enter your password: ")
    role = input("Enter your role (admin/user): ")
    new_accounts, message = register(accounts, nim, password, role)
    print(message)
    return new_accounts

@log_operation
def add_profile_wrapper(personnel_data, unit_data, nim):
    name = input("Enter your name: ")
    rank = input("Enter your rank: ")
    try:
        unit_id = int(input("Enter your Unit ID (0 if unknown): "))
    except ValueError:
        print("Invalid Unit ID.")
        return personnel_data
    new_personnel_data, message = add_profile(personnel_data, nim, name, rank, unit_id, unit_data)
    print(message)
    return new_personnel_data

@log_operation
def login_wrapper(accounts):
    nim = input("Enter your NIM: ")
    password = input("Enter your password: ")
    nim, message = login(accounts, nim, password)
    print(message)
    return nim

@log_operation
def update_profile_wrapper(personnel_data, unit_data, nim):
    new_name = input("Enter new name (leave blank to keep current): ") or None
    new_rank = input("Enter new rank (leave blank to keep current): ") or None
    new_unit_id = input("Enter new Unit ID (leave blank to keep current): ") or None
    new_unit_id = int(new_unit_id) if new_unit_id else None
    personnel_data, message = update_profile(personnel_data, nim, new_name, new_rank, new_unit_id, unit_data)
    print(message)
    return personnel_data

# ======================
# Closure (Unit management example)
# ======================

def manage_units(unit_data):
    # Closure to manage unit additions
    def add_unit_with_closure(unit_id, unit_name):
        nonlocal unit_data  # Access the unit_data variable from outer scope
        if any(u["unit_id"] == unit_id for u in unit_data):
            return unit_data, f"Unit ID {unit_id} already exists."
        unit_data.append({"unit_id": unit_id, "unit_name": unit_name})
        return unit_data, f"Unit {unit_name} added successfully."
    
    return add_unit_with_closure

# Create a closure function for unit management
unit_manager = manage_units(unit_data)

# ======================
# Menu dan Main Loop
# ======================

def user_menu(nim, accounts, personnel_data, unit_data):
    while True:
        print("\nUser Menu:")
        print("1. View Profile")
        print("2. Update Profile")
        print("3. Delete Profile")
        print("0. Logout")
        choice = input("Choose an option: ")
        if choice == "1":
            print(view_profile(personnel_data, unit_data, nim))
        elif choice == "2":
            personnel_data = update_profile_wrapper(personnel_data, unit_data, nim)
        elif choice == "3":
            confirm = input("Are you sure you want to delete your profile? (yes/no): ")
            if confirm.lower() == 'yes':
                personnel_data, accounts, message = delete_profile(personnel_data, accounts, nim)
                print(message)
                break
        elif choice == "0":
            break
        else:
            print("Invalid choice.")

def main():
    global accounts, personnel_data, unit_data

    while True:
        print("\nMain Menu:")
        print("1. Register")
        print("2. Login")
        print("0. Exit")
        choice = input("Choose an option: ")
        if choice == "1":
            accounts = register_wrapper(accounts)
        elif choice == "2":
            nim = login_wrapper(accounts)
            if nim:
                user_menu(nim, accounts, personnel_data, unit_data)
        elif choice == "0":
            break
        else:
            print("Invalid choice.")

main()



Main Menu:
1. Register
2. Login
0. Exit


Choose an option:  0
